In [95]:
#r "nuget:Microsoft.ML"

Installing package Microsoft.ML..done!

Successfully added reference to package Microsoft.ML, version

In [96]:
using System;
using System.IO;
using System.Linq;
using Microsoft.ML;
using Microsoft.ML.Data;
using XPlot.Plotly;

In [97]:
display(h1("Análise dos dados da COVID-19"));
display(h3("Base utilizada: brasil.io"));
display(h4("Criando classe para leitura dos dados:"));

Análise dos dados da COVID-19

Base utilizada: brasil.io

Criando classe para leitura dos dados:

In [120]:
/* Classe para leitura dos dados */
public class CovidCase {
    [LoadColumn(0)] public DateTime Date {get; set;}
    [LoadColumn(1)] public string State {get ;set;}
    [LoadColumn(2)] public string City {get; set;}
    [LoadColumn(3)] public string PlaceType {get; set;}
    [LoadColumn(4)] public UInt64 Confirmed {get; set;}
    [LoadColumn(5)] public UInt64 Deaths {get; set;}
    [LoadColumn(6)] public UInt64 OrderForPlace {get; set;}
    [LoadColumn(7)] public bool IsLast {get; set;}
    [LoadColumn(8)] public UInt64 EstimatedPopulation2019 {get; set;}
    [LoadColumn(9)] public UInt32 CityIBGEcode {get; set;}
    [LoadColumn(10)] public double ConfirmedPer100kHabitants {get; set;}
    [LoadColumn(11)] public double DeathRate {get; set;}
}

public class TotalEmpresas {
    [LoadColumn(0)] public string Estado {get; set;}
    [LoadColumn(1)] public UInt32 CodigoIbge {get ;set;}
    [LoadColumn(2)] public string Municipio {get; set;}
    [LoadColumn(3)] public UInt32 Quantidade {get; set;}    
}

public class Match {
    public string Estado {get; set;}
    public UInt32 CodigoIbge {get ;set;}
    public string Municipio {get; set;}
    public UInt32 QuantidadeEmpresas {get; set;} 
    public UInt64 Confirmados {get; set;} 
    public UInt64 Mortos {get; set;} 
}

In [121]:
display(h4("Leitura dos dados:"));

Leitura dos dados:

In [132]:
/* Nome do arquivo */
var casosCovid = Path.Combine(Environment.CurrentDirectory, "casos.csv");
var totalEmpresas = Path.Combine(Environment.CurrentDirectory, "totalEmpresas.csv");

/* Machine Learning context */
var context = new MLContext();


var dataCasosCovid = context.Data.LoadFromTextFile<CovidCase>(
    path: casosCovid,
    hasHeader: true,
    separatorChar: ','
);

var datatotalEmpresas = context.Data.LoadFromTextFile<TotalEmpresas>(
    path: totalEmpresas,
    hasHeader: true,
    separatorChar: ';'
);

/* Array de dados do arquivo CSV */
var cases = context.Data.CreateEnumerable<CovidCase>(
    dataCasosCovid,
    reuseRowObject: false)
    .ToArray();

var empresas = context.Data.CreateEnumerable<TotalEmpresas>(
    datatotalEmpresas,
    reuseRowObject: false)
    .ToArray();
    
/* Resgata os casos de Juiz de Fora */
var jfCases = cases.Where(x => x.City == "Juiz de Fora");
var jfEmpresas = empresas.Where(x => x.Municipio  == "Juiz de Fora");


In [92]:
display(h4("Gráfico:"));

Gráfico:

In [131]:
var confirmedJF = new Graph.Bar()
{
    x = jfCases.Select(x => x.Date).ToArray(),
    y = jfCases.Select(x => x.Confirmed).ToArray(),
    name = "Confirmados"
};

var deathsJf = new Graph.Bar()
{
    x = jfCases.Select(x => x.Date).ToArray(),
    y = jfCases.Select(x => x.Deaths).ToArray(),
    name = "Mortes"
};

var empresasJf = new Graph.Bar()
{
    x = jfEmpresas.Select(x => x.Municipio),
    y = jfEmpresas.Select(x => x.Quantidade),
    name = "Empresas"
};


var chart = Chart.Plot(new[] {confirmedJF, deathsJf});
var layout = new Layout.Layout(){ barmode = "group", title="Número de casos confirmados e mortes COVID-19 - Juiz de Fora" };
chart.WithLayout(layout);
chart.WithXTitle("Data");
chart.WithYTitle("Número de casos Confirmados");
chart.WithLegend(true);
chart.Width = 700;
chart.Height = 400;

display(chart);

var chart1 = Chart.Plot(new[] {empresasJf});
var layout1 = new Layout.Layout(){ barmode = "group", title="Total de Empresas - Juiz de Fora" };
chart.WithLayout(layout1);
chart.WithLegend(true);
chart.Width = 700;
chart.Height = 400;

display(chart1);


In [56]:
display(h4("Análise das top 10 cidades com mais casos por habitante"));

Análise das top 10 cidades com mais casos por habitante

In [124]:
/* Selecionando os dados das maiores cidades com casos por número de covid */

var topCidades= cases.Where(x => x.IsLast && x.PlaceType == "city" && x.EstimatedPopulation2019 > 100000).OrderByDescending(x => x.ConfirmedPer100kHabitants).Take(10);


In [58]:
display(h4(" Gráfico Top 10 cidades com maiores números de casos por 100k de habitantes"));

Gráfico Top 10 cidades com maiores números de casos por 100k de habitantes

In [59]:
var confirmedJF = new Graph.Bar()
{
    x = topCidades.Select(x => x.City).ToArray(),
    y = topCidades.Select(x => x.Confirmed).ToArray(),
    name = "Confirmados"
};

var confirmedJF1 = new Graph.Bar()
{
    x = topCidades.Select(x => x.City).ToArray(),
    y = topCidades.Select(x => x.Deaths).ToArray(),
    name = "Mortes"
};

var chart = Chart.Plot(new[] { confirmedJF,confirmedJF1 });
var layout = new Layout.Layout(){ barmode = "group", title="Top 10 cidades com maiores números de casos por 100k de habitantes" };
chart.WithLayout(layout);
chart.WithXTitle("Cidades");
chart.WithYTitle("Número de casos Confirmados");
chart.WithLegend(true);
chart.Width = 700;
chart.Height = 400;

display(chart);

In [133]:
display(h4("Match das cidades e empresas"));

Match das cidades e empresas

In [ ]:
/* Match da cidades */

